In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
from collections import defaultdict
from IPython.display import display_html
from itertools import chain, cycle
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import *
from src.util import *
from src import config
from src.visualize import *
pd.set_option('display.float_format', lambda x: '%.04f' % x)
gezi.set_pandas_widder()

No module named 'tensorflow.python.keras.layers.preprocessing'


In [2]:
d = pd.read_feather('../input/feedback-prize-2021/train_en.fea')

In [3]:
d['num_words'] = d.text.apply(lambda x: len(x.split()))

In [4]:
d.groupby('kfold')['num_words'].agg('mean').reset_index()

,kfold,num_words
0,0,466.9746
1,1,469.8603
2,2,465.2151
3,3,464.8431
4,4,476.1696


In [5]:
d.groupby('kfold')['num_words'].agg('max').reset_index()

,kfold,num_words
0,0,1301
1,1,1656
2,2,1228
3,3,1278
4,4,1401


In [6]:
dg = d.groupby('id')['para_type'].agg(list).reset_index()
dg

,id,para_type
0,0000D23A521A,"[3, 2, 2, 1, 6, 7, 2, 4]"
1,00066EA9880D,"[5, 3, 0, 1, 2, 1, 2, 0, 1, 2, 4]"
2,000E6DE9E817,"[0, 3, 0, 6, 7, 2, 0, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 4, 0]"
3,001552828BD0,"[5, 2, 1, 1, 2, 1, 2, 1, 2, 3]"
4,0016926B079C,"[3, 1, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2]"
...,...,...
15589,FFF1442D6698,"[5, 3, 1, 1, 0, 2, 0, 2, 6, 7, 0, 2, 0, 4]"
15590,FFF1ED4F8544,"[5, 3, 1, 1, 1, 1, 2, 1, 2]"
15591,FFF868E06176,"[5, 3, 0, 1, 2, 0, 1, 2, 1, 2, 4]"
15592,FFFD0AF13501,"[0, 1, 1, 2, 1, 1, 2, 0, 3, 4]"


In [7]:
dg['dis_types'] = dg.para_type.apply(lambda l: ['Start', *(id2dis[x] for x in l), 'End'])
dg

,id,para_type,dis_types
0,0000D23A521A,"[3, 2, 2, 1, 6, 7, 2, 4]","[Start, Position, Evidence, Evidence, Claim, Counterclaim, Rebuttal, Evidence, Concluding Statement, End]"
1,00066EA9880D,"[5, 3, 0, 1, 2, 1, 2, 0, 1, 2, 4]","[Start, Lead, Position, Nothing, Claim, Evidence, Claim, Evidence, Nothing, Claim, Evidence, Concluding Statement, End]"
2,000E6DE9E817,"[0, 3, 0, 6, 7, 2, 0, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 4, 0]","[Start, Nothing, Position, Nothing, Counterclaim, Rebuttal, Evidence, Nothing, Claim, Evidence, Claim, Nothing, Claim, Nothing, Claim, Nothing, Claim, Nothing, Evidence, Concluding Statement, Nothing, End]"
3,001552828BD0,"[5, 2, 1, 1, 2, 1, 2, 1, 2, 3]","[Start, Lead, Evidence, Claim, Claim, Evidence, Claim, Evidence, Claim, Evidence, Position, End]"
4,0016926B079C,"[3, 1, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2]","[Start, Position, Claim, Claim, Nothing, Claim, Claim, Evidence, Claim, Evidence, Claim, Claim, Evidence, End]"
...,...,...,...
15589,FFF1442D6698,"[5, 3, 1, 1, 0, 2, 0, 2, 6, 7, 0, 2, 0, 4]","[Start, Lead, Position, Claim, Claim, Nothing, Evidence, Nothing, Evidence, Counterclaim, Rebuttal, Nothing, Evidence, Nothing, Concluding Statement, End]"
15590,FFF1ED4F8544,"[5, 3, 1, 1, 1, 1, 2, 1, 2]","[Start, Lead, Position, Claim, Claim, Claim, Claim, Evidence, Claim, Evidence, End]"
15591,FFF868E06176,"[5, 3, 0, 1, 2, 0, 1, 2, 1, 2, 4]","[Start, Lead, Position, Nothing, Claim, Evidence, Nothing, Claim, Evidence, Claim, Evidence, Concluding Statement, End]"
15592,FFFD0AF13501,"[0, 1, 1, 2, 1, 1, 2, 0, 3, 4]","[Start, Nothing, Claim, Claim, Evidence, Claim, Claim, Evidence, Nothing, Position, Concluding Statement, End]"


In [8]:
dg['num_paras'] = dg.para_type.apply(len)

In [9]:
dg.describe([.25,.5,.75,.9,.99,.999])

,num_paras
count,15594.0000
mean,11.2325
std,3.9939
min,1.0000
25%,8.0000
50%,11.0000
75%,14.0000
90%,16.0000
99%,21.0000
99.9%,26.0000


In [10]:
def has_adjacent(dis_types, dis):
  pre_dis = None
  for dis_ in dis_types:
    if dis_ == dis and dis_ == pre_dis:
      return 1
    pre_dis = dis_
  return 0

In [11]:
def adjacent_count(dis_types, dis):
  pre_dis = None
  count = 0
  for dis_ in dis_types:
    if dis_ == dis and dis_ == pre_dis:
      count += 1
    pre_dis = dis_
  return count

In [12]:
for dis in ALL_CLASSES:
  dg[f'adj_{dis}'] = dg['dis_types'].apply(lambda x: has_adjacent(x, dis))
  dg[f'count_{dis}'] = dg['dis_types'].apply(lambda x: x.count(dis))
  dg[f'adjcount_{dis}'] = dg['dis_types'].apply(lambda x: adjacent_count(x, dis))

In [13]:
dg.describe([.25,.5,.75,.9,.99,.995,.999])

,num_paras,adj_Nothing,count_Nothing,adjcount_Nothing,adj_Claim,count_Claim,adjcount_Claim,adj_Evidence,count_Evidence,adjcount_Evidence,adj_Position,count_Position,adjcount_Position,adj_Concluding Statement,count_Concluding Statement,adjcount_Concluding Statement,adj_Lead,count_Lead,adjcount_Lead,adj_Counterclaim,count_Counterclaim,adjcount_Counterclaim,adj_Rebuttal,count_Rebuttal,adjcount_Rebuttal
count,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000,15594.0000
mean,11.2325,0.0000,1.9798,0.0000,0.4195,3.2193,0.6611,0.2197,2.9307,0.2752,0.0001,0.9888,0.0001,0.0001,0.8660,0.0002,0.0001,0.5967,0.0001,0.0037,0.3730,0.0042,0.0006,0.2781,0.0007
std,3.9939,0.0000,1.9346,0.0000,0.4935,1.7641,0.9213,0.4141,1.1755,0.5835,0.0080,0.1338,0.0080,0.0080,0.3572,0.0240,0.0080,0.4911,0.0080,0.0604,0.6656,0.0741,0.0253,0.5669,0.0289
min,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,8.0000,0.0000,0.0000,0.0000,0.0000,2.0000,0.0000,0.0000,2.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,11.0000,0.0000,2.0000,0.0000,0.0000,3.0000,0.0000,0.0000,3.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
75%,14.0000,0.0000,3.0000,0.0000,1.0000,4.0000,1.0000,0.0000,3.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000
90%,16.0000,0.0000,5.0000,0.0000,1.0000,6.0000,2.0000,1.0000,4.0000,1.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000
99%,21.0000,0.0000,7.0000,0.0000,1.0000,7.0000,3.0000,1.0000,6.0000,2.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,3.0000,0.0000,0.0000,2.0000,0.0000
99.5%,22.0000,0.0000,8.0000,0.0000,1.0000,8.0000,4.0000,1.0000,7.0000,3.0000,0.0000,1.0000,0.0000,0.0000,2.0000,0.0000,0.0000,1.0000,0.0000,0.0000,3.0000,0.0000,0.0000,3.0000,0.0000


In [14]:
dg[(dg.count_Rebuttal>0) & (dg.count_Counterclaim==0)]

,id,para_type,dis_types,num_paras,adj_Nothing,count_Nothing,adjcount_Nothing,adj_Claim,count_Claim,adjcount_Claim,adj_Evidence,count_Evidence,adjcount_Evidence,adj_Position,count_Position,adjcount_Position,adj_Concluding Statement,count_Concluding Statement,adjcount_Concluding Statement,adj_Lead,count_Lead,adjcount_Lead,adj_Counterclaim,count_Counterclaim,adjcount_Counterclaim,adj_Rebuttal,count_Rebuttal,adjcount_Rebuttal
729,0C2CD7E1B791,"[0, 3, 0, 1, 1, 1, 1, 7, 4]","[Start, Nothing, Position, Nothing, Claim, Claim, Claim, Claim, Rebuttal, Concluding Statement, End]",9,0,2,0,1,4,3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
14832,F3CE7992E75E,"[0, 3, 1, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 2, 0, 1, 2, 0, 1, 0, 7, 0, 1, 2, 4]","[Start, Nothing, Position, Claim, Evidence, Nothing, Claim, Nothing, Claim, Nothing, Claim, Evidence, Evidence, Nothing, Claim, Evidence, Nothing, Claim, Evidence, Nothing, Claim, Nothing, Rebuttal, Nothing, Claim, Evidence, Concluding Statement, End]",26,0,9,0,0,8,0,1,6,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0


In [15]:
dg[(dg.count_Counterclaim>0) & (dg.count_Claim==0)]

,id,para_type,dis_types,num_paras,adj_Nothing,count_Nothing,adjcount_Nothing,adj_Claim,count_Claim,adjcount_Claim,adj_Evidence,count_Evidence,adjcount_Evidence,adj_Position,count_Position,adjcount_Position,adj_Concluding Statement,count_Concluding Statement,adjcount_Concluding Statement,adj_Lead,count_Lead,adjcount_Lead,adj_Counterclaim,count_Counterclaim,adjcount_Counterclaim,adj_Rebuttal,count_Rebuttal,adjcount_Rebuttal
58,0114B2C51347,"[0, 5, 3, 6, 7, 6, 7, 4, 0]","[Start, Nothing, Lead, Position, Counterclaim, Rebuttal, Counterclaim, Rebuttal, Concluding Statement, Nothing, End]",9,0,2,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,2,0,0,2,0
129,0226E802EEEA,"[5, 3, 6, 7, 2, 6, 7, 2, 6, 7, 2, 4]","[Start, Lead, Position, Counterclaim, Rebuttal, Evidence, Counterclaim, Rebuttal, Evidence, Counterclaim, Rebuttal, Evidence, Concluding Statement, End]",12,0,0,0,0,0,0,0,3,0,0,1,0,0,1,0,0,1,0,0,3,0,0,3,0
197,036F8E1EF7A2,"[2, 6, 7, 2, 3, 2]","[Start, Evidence, Counterclaim, Rebuttal, Evidence, Position, Evidence, End]",6,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
285,04DEFB0B11C5,"[5, 3, 2, 6, 2]","[Start, Lead, Position, Evidence, Counterclaim, Evidence, End]",5,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
330,05A474FBA68F,"[0, 3, 2, 0, 6, 0, 7, 0, 2, 0]","[Start, Nothing, Position, Evidence, Nothing, Counterclaim, Nothing, Rebuttal, Nothing, Evidence, Nothing, End]",10,0,5,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14666,F0E863BDEB82,"[3, 2, 6, 7, 2, 4]","[Start, Position, Evidence, Counterclaim, Rebuttal, Evidence, Concluding Statement, End]",6,0,0,0,0,0,0,0,2,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
14845,F3F68CD25F7A,"[3, 2, 6, 2, 4]","[Start, Position, Evidence, Counterclaim, Evidence, Concluding Statement, End]",5,0,0,0,0,0,0,0,2,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
15171,F9368F6BAB62,"[3, 2, 2, 2, 2, 6, 7]","[Start, Position, Evidence, Evidence, Evidence, Evidence, Counterclaim, Rebuttal, End]",7,0,0,0,0,0,0,1,4,3,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
15218,F9FA6D6CF768,"[3, 2, 6, 7, 4]","[Start, Position, Evidence, Counterclaim, Rebuttal, Concluding Statement, End]",5,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0


In [16]:
def count_unigram():
  counter = Counter()
  for row in tqdm(dg.itertuples(), total=len(dg)):
    for dis in row.dis_types:
      counter[dis] += 1
  return counter

In [17]:
counter = count_unigram()

  0%|          | 0/15594 [00:00<?, ?it/s]

In [18]:
counter

Counter({'Start': 15594,
         'Position': 15419,
         'Evidence': 45702,
         'Claim': 50202,
         'Counterclaim': 5817,
         'Rebuttal': 4337,
         'Concluding Statement': 13505,
         'End': 15594,
         'Lead': 9305,
         'Nothing': 30873})

In [25]:
dg.dis_types

0        [Start, Position, Evidence, Evidence, Claim, Counterclaim, Rebuttal, Evidence, Concluding Statement, End]                                                                                                    
1        [Start, Lead, Position, Nothing, Claim, Evidence, Claim, Evidence, Nothing, Claim, Evidence, Concluding Statement, End]                                                                                      
2        [Start, Nothing, Position, Nothing, Counterclaim, Rebuttal, Evidence, Nothing, Claim, Evidence, Claim, Nothing, Claim, Nothing, Claim, Nothing, Claim, Nothing, Evidence, Concluding Statement, Nothing, End]
3        [Start, Lead, Evidence, Claim, Claim, Evidence, Claim, Evidence, Claim, Evidence, Position, End]                                                                                                             
4        [Start, Position, Claim, Claim, Nothing, Claim, Claim, Evidence, Claim, Evidence, Claim, Claim, Evidence, End]                     

In [26]:
def count_bigrams():
  counter = Counter()
  for row in tqdm(dg.itertuples(), total=len(dg)):
    for i in range(0, len(row.dis_types) - 1):
      counter['|'.join(row.dis_types[i:i + 2])] += 1
  return counter

In [27]:
bi_counter = count_bigrams()

  0%|          | 0/15594 [00:00<?, ?it/s]

In [28]:
bi_counter

Counter({'Start|Position': 3593,
         'Position|Evidence': 2673,
         'Evidence|Evidence': 4291,
         'Evidence|Claim': 17731,
         'Claim|Counterclaim': 1024,
         'Counterclaim|Rebuttal': 3595,
         'Rebuttal|Evidence': 1801,
         'Evidence|Concluding Statement': 8715,
         'Concluding Statement|End': 10908,
         'Start|Lead': 7600,
         'Lead|Position': 6876,
         'Position|Nothing': 4312,
         'Nothing|Claim': 12999,
         'Claim|Evidence': 28095,
         'Evidence|Nothing': 9178,
         'Start|Nothing': 4003,
         'Nothing|Position': 3413,
         'Nothing|Counterclaim': 538,
         'Claim|Nothing': 9284,
         'Nothing|Evidence': 6867,
         'Concluding Statement|Nothing': 2159,
         'Nothing|End': 2813,
         'Lead|Evidence': 411,
         'Claim|Claim': 10309,
         'Evidence|Position': 764,
         'Position|End': 431,
         'Position|Claim': 7170,
         'Evidence|End': 1177,
         'Start|Cl

In [22]:
len(dg)

15594

In [23]:
dg['para_str'] = dg.para_type.apply(lambda l: ' '.join([str(x) for x in l]))

In [24]:
len(dg.para_str.unique())

9878